# Desafio - Aula 3

## 1) Crie uma classe de tratamento de texto. A classe deverá conter:
   ## -> remoção de números
   ## -> passar o texto para caixa baixa
   ## -> remoção de caracteres especiais
   ## -> remoção de stop-words
   ## -> Stemização/lemmatização (deve ser passado como parâmetro qual abordagem utilizar)

In [2]:
!pip install spacy
!python -m spacy download pt_core_news_sm
!python -m spacy download en

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


[+] Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[+] Linking successful
C:\Anaconda3\lib\site-packages\en_core_web_sm -->
C:\Anaconda3\lib\site-packages\spacy\data\en
You can now load the model via spacy.load('en')


You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
Você não tem privilégios suficientes para realizar esta operação.


In [198]:
import re
import nltk
import spacy
import nltk
from nltk.stem import PorterStemmer
import pandas as pd
from nltk.stem.rslp import RSLPStemmer
import re

STEM = "Stemizacao"
LEM = "Lemmatizacao"
    
class CleanText:
    
    def __init__(self, lan = 'english', norm = None, load_v = None, clean_regex = "([aA-zZ]+)", stemmer = RSLPStemmer()):
        nltk.download('stopwords', quiet = True)
        self.regex = clean_regex
        self.stops = nltk.corpus.stopwords.words(lan)
        
        if load_v == None:
            if lan == "english":
                self.nlp = spacy.load('en')
            elif lan == "portuguese":
                self.nlp = spacy.load('pt_core_news_sm')
            else:
                raise("idioma invalido")
        else:
            self.nlp = spacy.load(load_v)
        self.ps = stemmer
        
        if (norm != STEM) and (norm != LEM) and (norm != None):
            raise("Normalizacao invalida")
        self.norm = norm
        
    def clean(self, df, stops = None):
        if stops != None:
            self.stops = stops
            
        return df.apply(self.remove_special_char).apply(lambda x: self.remove_stop_words(x, self.stops)).apply(self.normalize)
        
    def remove_special_char(self, text):
        return " ".join(["".join(re.findall(self.regex, word)).lower() for word in text.split(" ")])
    
    def remove_stop_words(self, text, stops):
        if stops != None:
            return " ".join([word if word not in self.stops else "" for word in text.split(" ")])
        else:
            return text
        
    def normalize(self, text):
        text = re.sub(' +', ' ', text)
        text = re.sub('^ | $', '', text)
        if len(text) > 0:
            if self.norm == LEM:
                doc = self.nlp(text)
                return " ".join([token.lemma_ for token in doc])
            elif self.norm == STEM:
                return " ".join([self.ps.stem(word) for word in text.split(" ")])
        return text
    
d = {"a": ["I World1!.@ é223 is dogs  went go goes daspd% did conected", "hell0o world!"]}
df = pd.DataFrame(d)
clean_text = CleanText(norm = LEM, lan = "english")
clean_text.clean(df['a'])

0    world dog go go go daspd conecte
1                         hello world
Name: a, dtype: object

## 2) Refaça o exercício de aula (movie_review) realizando o tratamento do texto antes. Houve diferença? Descreva

In [175]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

data = pd.read_csv('movie_review1.csv',index_col=0)

In [183]:
vectfidf = TfidfVectorizer()
vector = vectfidf.fit_transform(data['text'])
X_train, X_test, y_train, y_test = train_test_split(vector, data['tag'], test_size=0.3, random_state=69)
model = MultinomialNB().fit(X_train, y_train)
predicted = model.predict(X_test)
print("Antes: ")
print(classification_report(y_test, predicted))
print("\nacc: %f" % np.mean(predicted == y_test))

Antes: 
             precision    recall  f1-score   support

          0       0.72      0.67      0.69      9561
          1       0.70      0.74      0.72      9855

avg / total       0.71      0.71      0.71     19416


acc: 0.706737


In [201]:
vectfidf = TfidfVectorizer()
data['text'] = CleanText().clean(data['text'])
vector = vectfidf.fit_transform(data['text'])
X_train, X_test, y_train, y_test = train_test_split(vector, data['tag'], test_size=0.3, random_state=69)
model = MultinomialNB().fit(X_train, y_train)
predicted = model.predict(X_test)
print("Depois: ")
print(classification_report(y_test, predicted))
print("\nacc: %f" % np.mean(predicted == y_test))

Depois: 
             precision    recall  f1-score   support

          0       0.71      0.65      0.68      9561
          1       0.69      0.74      0.72      9855

avg / total       0.70      0.70      0.70     19416


acc: 0.698754


### R: Não foi possivel observar uma melhora utilizando a classe de tratamento de texto

## 3) Utilizando o dataset disposto no portal, faça:

### -> extraia o dataset na pasta do notebook
### -> crie uma função que leia o conteúdo de cada uma das pastas, amazene num dataframe com duas colunas (review, tag)

In [202]:
# resposta 3
import os
import numpy as np
import pandas as pd

def read_reviews(directory):
    return pd.concat([read_reviews_from_path(os.path.join(directory, 'pos'), 1), read_reviews_from_path(os.path.join(directory, 'neg'), 0)]).reset_index(drop = True)

def read_reviews_from_path(directory, tag, review_column = 'review', tag_column = 'tag'):
    df = pd.DataFrame({review_column: [], tag_column: []})
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            f = open(os.path.join(directory, filename))
            lines = f.read()
            df = df.append({review_column: lines, tag_column: tag}, ignore_index = True)
            continue
        else:
            continue
    return df

In [203]:
df = read_reviews('txt_sentoken')

   ### -> utilize a classe de tratamento de texto criada acima para tratar o texto
   ### -> crie um pipeline de classificação de texto (countvectorizer/tfidfvectorizer,divisão em treino/teste,instância de modelo, fit e predict)
   ### -> imprima o relatório de classificação
   ### -> OBS: teste várias opções de stemming/lemmatizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

class Pipeline:
    
    def __init__(self, vectorizer = CountVectorizer(), model = MultinomialNB(), cleaner = CleanText()):
        self.vectorizer = vectorizer
        self.model = model
        self.cleaner = cleaner
    def fit(self, X, y, test_size = 0.3):
        X = self.cleaner.clean(X)
        X = self.vectorizer.fit_transform(X)
        X_test, X_train, y_test, y_train = train_test_split(X, y, test_size = test_size, random_state = 69, shuffle = True)
        self.model.fit(X_train, y_train)
        y_pred = self.predict(X_test)
        self.reports(y_test, y_pred)
        
    def predict(self, X):
        return self.model.predict(X)
    
    def reports(self, y_true, y_pred):
        print("Classification Report: \n")
        print(classification_report(y_true, y_pred))
        print("Confusion Matrix: \n")
        print(confusion_matrix(y_true, y_pred))

In [ ]:
print("************tf-idf/sem normalizacao***********\n")
pipeline = Pipeline(vectorizer = TfidfVectorizer())
pipeline.fit(df['review'], df['tag'])
print("**********************************************\n")
print("*************tf-idf/stemmização***************\n")
pipeline = Pipeline(vectorizer = TfidfVectorizer(), cleaner = CleanText(norm = STEM))
pipeline.fit(df['review'], df['tag'])
print("**********************************************\n")
print("*************tf-idf/lemmatização**************\n")
pipeline = Pipeline(vectorizer = TfidfVectorizer(), cleaner = CleanText(norm = LEM))
pipeline.fit(df['review'], df['tag'])
print("**********************************************\n")
print("*******count vectorizer/sem normalizacao******\n")
pipeline = Pipeline(vectorizer = CountVectorizer())
pipeline.fit(df['review'], df['tag'])
print("**********************************************\n")
print("********count vectorizer/stemização***********\n")
pipeline = Pipeline(vectorizer = CountVectorizer(), cleaner = CleanText(norm = STEM))
pipeline.fit(df['review'], df['tag'])
print("**********************************************\n")
print("********count vectorizer/lemmatização*********\n")
pipeline = Pipeline(vectorizer = CountVectorizer(), cleaner = CleanText(norm = LEM))
pipeline.fit(df['review'], df['tag'])
print("**********************************************\n")

************tf-idf/sem normalizacao***********

Classification Report: 

             precision    recall  f1-score   support

        0.0       0.89      0.43      0.58       713
        1.0       0.61      0.94      0.74       687

avg / total       0.75      0.68      0.66      1400

Confusion Matrix: 

[[305 408]
 [ 39 648]]
**********************************************

*************tf-idf/stemmização***************

